<a href="https://colab.research.google.com/github/Fabiorsi1979/DS-Comunidade/blob/main/SQL_basico_Op_Logicos_intervalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operadores Lógicos de Intervalo

#### Os operadores de intervalo selecionam as linhas dsa tabela dentro de um intervalo de valor que podem ser: números, textos e datas.

#### **BETWEEN** - quando a consulta é necessária para um intervalo numérico:
#### **IN** - filtra exatamente os registros que contenham as informações solicitadas
#### **LIKE** - filtra todoas as categorias caso encontre o padrao no valor da coluna
#### **HAVING** - o operador Having é usado para filtrar linhas a partir do resultado de uma função agregadora quando deve ser aplicada a clausula Where

# Exercícios

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
path = '/content/drive/MyDrive/db_olist.sqlite'

In [8]:
import sqlite3
import pandas as pd

In [9]:
def sql_logico_intervalo(*args):
  conexao = sqlite3.connect(path)
  cursor = conexao.cursor()

  cursor.execute(consulta)
  dados = cursor.fetchall()

  cursor.close()
  conexao.close()

  sql_consulta = pd.DataFrame(data=dados, columns=colunas)
  return sql_consulta

#### 1. Quantos clientes únicos tiveram seu pedidos com status de “processing”, “shipped” e “delivered”, feitos entre os dias 01 e 31 de Outubro de 2016. Mostrar o resultado somente se o número total de clientes for acima de 5.

In [31]:
consulta = """
  SELECT
      o.order_status,
      COUNT(DISTINCT o.customer_id)
  FROM orders o WHERE o.order_status IN ('processing','shipped','delivered')
      AND o.order_approved_at BETWEEN '2016-10-01' AND '2016-10-31'
  GROUP BY o.order_status
  HAVING COUNT(DISTINCT o.customer_id)>5
"""
colunas =['Status','Qtde_Clientes']

exercicio1 = sql_logico_intervalo(consulta,colunas)
exercicio1

,Status,Qtde_Clientes
0,delivered,265
1,shipped,9


#### 2. Mostre a quantidade total dos pedidos e o valor total do pagamento, para pagamentos entre 1 e 5 prestações ou um valor de pagamento acima de R$ 5000.

In [32]:
consulta = """
  SELECT
    COUNT(op.order_id),
    SUM(op.payment_value)
  FROM order_payments op
  WHERE (op.payment_installments BETWEEN 1 AND 5 ) OR op.payment_value>5000
  """
colunas =["Total Pedidos", "Valor Total"]

exercicio2 = sql_logico_intervalo(consulta,colunas)
exercicio2

,Total Pedidos,Valor Total
0,87758,1.110963e+07


#### 3.  Quantos produtos estão cadastrados nas categorias: perfumaria, brinquedos, esporte lazer e cama mesa, que possuem entre 5 e 10 fotos, um peso que não está entre 1 e 5 g, um altura maior que 10 cm, uma largura maior que 20 cm. Mostra somente as linhas com mais de 10 produtos únicos.

In [33]:
consulta = """
  SELECT p.product_category_name,
    COUNT(DISTINCT p.product_id)
  FROM products p
  WHERE p.product_category_name IN ('perfumaria','brinquedos','esporte_lazer','cama_mesa_banho')
    AND (p.product_photos_qty BETWEEN 5 AND 10 AND p.product_weight_g NOT BETWEEN 1 AND 5
    AND product_height_cm > 10 AND product_length_cm > 20)
  GROUP BY p.product_category_name HAVING COUNT(DISTINCT p.product_id)>10
  """
colunas = ["Categoria","Qtde produtos cadastrados"]

exercicio3 = sql_logico_intervalo(consulta,colunas)
exercicio3

,Categoria,Qtde produtos cadastrados
0,brinquedos,84
1,cama_mesa_banho,36
2,esporte_lazer,182


#### 4. Refazer a consulta SQL abaixo, usando os operadores de intervalo.
~~~python
SELECT
    order_status ,
    COUNT( order_id ) AS pedidos
FROM orders o
WHERE ( order_status = 'processing' OR order_status = 'canceled' )
 AND ( o.order_estimated_delivery_date > '2017-01-01'
 AND o.order_estimated_delivery_date < '2017-12-31' )
GROUP BY order_status
~~~

In [34]:
consulta ="""
  SELECT
    order_status,
    COUNT (o.order_id)
  FROM orders o
  WHERE ( o.order_status = 'processing' OR order_status = 'canceled' )
    AND (o.order_estimated_delivery_date BETWEEN '2017-01-01' AND '2017-12-31')
  GROUP BY o.order_status
  """
colunas =["Status","Pedidos"]

exercicio4 = sql_logico_intervalo(consulta,colunas)
exercicio4

,Status,Pedidos
0,canceled,258
1,processing,209


#### 5. Qual a quantidade de cidades únicas dos vendedores no estado de São Paulo ou Rio de Janeiro com a latitude maior que -24.54 e longitude menor que -45.63?


In [29]:
consulta = """
SELECT
  g.geolocation_state,
  COUNT( DISTINCT g.geolocation_city ) AS cidades
FROM geolocation g
WHERE g.geolocation_state IN ( 'SP', 'RJ' )
 AND ( g.geolocation_lat > -24.54 AND g.geolocation_lng < -45.63 )
GROUP BY g.geolocation_state
"""
colunas =["UF","Qtde Cidades Vendedores"]

exercicio5 = sql_logico_intervalo(consulta,colunas)
exercicio5

,UF,Qtde Cidades Vendedores
0,RJ,9
1,SP,996


#### 6. Quantos produtos estão cadastrados em qualquer categorias que comece com a letra “a” e termine com a letra “o” e que possuem mais de 5 fotos? Mostrar as linhas com mais de 10 produtos.

In [35]:
consulta ="""
SELECT
  p.product_category_name,
  COUNT (DISTINCT p.product_id)
FROM products p
WHERE p.product_category_name LIKE 'a%o'
GROUP BY p.product_category_name
HAVING COUNT (DISTINCT p.product_id)>10
 """
colunas = ["Categoria","Qtde produtos"]

exercicio6 = sql_logico_intervalo(consulta,colunas)
exercicio6

,Categoria,Qtde produtos
0,agro_industria_e_comercio,74
1,artes_e_artesanato,19
2,audio,58
3,automotivo,1900


#### 7. Qual o número de clientes únicos, agrupados por estado e por cidades que comecem com a letra “m”, tem a letra “o” e terminem com a letra “a”? Mostrar os resultados somente para o número de clientes únicos maior que 10.


In [36]:
consulta ="""
SELECT
  c.customer_city,
  c.customer_state,
  COUNT (DISTINCT c.customer_id)
FROM customer c
WHERE c.customer_city LIKE 'm%o%a'
GROUP BY c.customer_state,c.customer_city
HAVING COUNT (DISTINCT c.customer_id)>10
 """
colunas = ["Cidade","UF","Qtde clientes"]

exercicio7 = sql_logico_intervalo(consulta,colunas)
exercicio7

,Cidade,UF,Qtde clientes
0,matias barbosa,MG,13
1,mairipora,SP,75
2,mococa,SP,34
3,mongagua,SP,42
4,monte azul paulista,SP,19
